# Roboflow API를 통한 방법

##환경 설정 및 라이브러리 설치

In [ ]:
!pip install inference-sdk
!pip install pytesseract
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!pip install opencv-python-headless
!pip install networkx

from google.colab import drive
drive.mount('/content/drive')

##Roboflow API 설정

https://roboflow.com/

In [ ]:
from inference_sdk import InferenceHTTPClient

API_KEY = "your_api_key_here"
CLIENT = InferenceHTTPClient(api_url="https://infer.roboflow.com", api_key=API_KEY)

##모델 ID 정의

In [ ]:
ROOM_MODEL_ID = "floor-plan-rendering/room-segmentation-model/1"  # 버전 확인 필요
WALL_MODEL_ID = "iiitbangalore/floor-plan-segmentation-dtr4r/1"  # 버전 확인 필요

##이미지 로드 및 인퍼런스 함수 정의

In [ ]:
# 이미지 로드 함수
import cv2
import numpy as np

def load_image(path):
    return cv2.imread(path)

In [ ]:
# 인퍼런스 실행 함수
def run_inference(image_path, model_id):
    result = CLIENT.infer(image_path, model_id=model_id)
    return result

##텍스트 추출 함수 정의

In [ ]:
import pytesseract

def extract_text(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text_data = pytesseract.image_to_data(gray, lang='kor', output_type=pytesseract.Output.DICT)
    return text_data

##방 번호 할당 함수 정의

In [ ]:
def assign_room_numbers(room_masks, text_data):
    room_labels = {}
    for i, mask in enumerate(room_masks):
        for j in range(len(text_data['text'])):
            if text_data['text'][j].strip():
                x = text_data['left'][j]
                y = text_data['top'][j]
                if mask[y, x]:  # 마스크 내 텍스트 확인
                    room_labels[i] = text_data['text'][j]
                    break
    return room_labels

##JSON 기반 그래프 로드 함수 정의

In [ ]:
import networkx as nx
import json

def load_graph_from_json(json_path, floor):
    with open(json_path, 'r') as f:
        data = json.load(f)
    G = nx.Graph()
    for item in data:
        if "id" in item:
            node_id = f"{floor}_{item['id']}"
            G.add_node(node_id, name=item['name'], type=item['type'], x=item['x'], y=item['y'], floor=floor)
        elif "source" in item:
            source = f"{floor}_{item['source']}"
            target = f"{floor}_{item['target']}"
            G.add_edge(source, target, weight=item['weight'])
    return G

##이미지 기반 그래프 생성 함수 정의

In [ ]:
def build_graph_from_image(image_path, floor):
    image = load_image(image_path)
    room_result = run_inference(image_path, ROOM_MODEL_ID)
    wall_result = run_inference(image_path, WALL_MODEL_ID)

    # 방 마스크 추출 (예시, 실제 결과 형식에 따라 조정 필요)
    room_masks = [pred['mask'] for pred in room_result['predictions'] if pred['class'] == 'room']
    wall_masks = [pred['mask'] for pred in wall_result['predictions'] if pred['class'] == 'wall']

    wall_mask = np.zeros(image.shape[:2], dtype=bool)
    for mask in wall_masks:
        wall_mask |= mask
    open_space_mask = ~wall_mask

    text_data = extract_text(image)
    room_labels = assign_room_numbers(room_masks, text_data)

    stair_positions = [(text_data['left'][i], text_data['top'][i], text) for i, text in enumerate(text_data['text']) if 'stair' in text.lower()]
    elevator_positions = [(text_data['left'][i], text_data['top'][i], text) for i, text in enumerate(text_data['text']) if 'elevator' in text.lower()]

    G = nx.Graph()

    # 방 노드 추가
    room_nodes = []
    for i, mask in enumerate(room_masks):
        M = cv2.moments(mask.astype(np.uint8))
        cx = int(M['m10'] / M['m00']) if M['m00'] != 0 else 0
        cy = int(M['m01'] / M['m00']) if M['m00'] != 0 else 0
        name = room_labels.get(i, f"Room_{i}")
        node_id = f"{floor}_room_{i}"
        G.add_node(node_id, name=name, type='Room', x=cx, y=cy, floor=floor)
        room_nodes.append(node_id)

    # 계단 및 엘리베이터 노드 추가
    for i, (x, y, text) in enumerate(stair_positions):
        node_id = f"{floor}_stair_{i}"
        G.add_node(node_id, name=text, type='Stair', x=x, y=y, floor=floor)
    for i, (x, y, text) in enumerate(elevator_positions):
        node_id = f"{floor}_elevator_{i}"
        G.add_node(node_id, name=text, type='Elevator', x=x, y=y, floor=floor)

    # 그리드 노드 추가
    grid_spacing = 20
    height, width = image.shape[:2]
    grid_nodes = []
    for i in range(0, height, grid_spacing):
        for j in range(0, width, grid_spacing):
            if open_space_mask[i, j]:
                node_id = f"{floor}_grid_{i}_{j}"
                G.add_node(node_id, name='Corridor', type='Corridor', x=j, y=i, floor=floor)
                grid_nodes.append(node_id)

    # 인접 그리드 노드 연결
    for node in grid_nodes:
        x = G.nodes[node]['x']
        y = G.nodes[node]['y']
        neighbors = [(x + grid_spacing, y), (x - grid_spacing, y), (x, y + grid_spacing), (x, y - grid_spacing)]
        for nx, ny in neighbors:
            if 0 <= ny < height and 0 <= nx < width and open_space_mask[ny, nx]:
                neighbor_id = f"{floor}_grid_{ny}_{nx}"
                if neighbor_id in G.nodes:
                    distance = np.sqrt((x - nx)**2 + (y - ny)**2)
                    G.add_edge(node, neighbor_id, weight=distance)

    # 방 노드와 가장 가까운 그리드 노드 연결
    for room_node in room_nodes:
        rx, ry = G.nodes[room_node]['x'], G.nodes[room_node]['y']
        min_distance = float('inf')
        nearest_grid = None
        for grid_node in grid_nodes:
            gx, gy = G.nodes[grid_node]['x'], G.nodes[grid_node]['y']
            distance = np.sqrt((rx - gx)**2 + (ry - gy)**2)
            if distance < min_distance:
                min_distance = distance
                nearest_grid = grid_node
        if nearest_grid:
            G.add_edge(room_node, nearest_grid, weight=min_distance)

    # 계단 및 엘리베이터 노드와 가장 가까운 그리드 노드 연결 (동일)
    for node_type in ['Stair', 'Elevator']:
        for node in [n for n in G.nodes if G.nodes[n]['type'] == node_type and G.nodes[n]['floor'] == floor]:
            x, y = G.nodes[node]['x'], G.nodes[node]['y']
            min_distance = float('inf')
            nearest_grid = None
            for grid_node in grid_nodes:
                gx, gy = G.nodes[grid_node]['x'], G.nodes[grid_node]['y']
                distance = np.sqrt((x - gx)**2 + (y - gy)**2)
                if distance < min_distance:
                    min_distance = distance
                    nearest_grid = grid_node
            if nearest_grid:
                G.add_edge(node, nearest_grid, weight=min_distance)

    # 그래프 저장
    with open(f'/content/drive/MyDrive/G{floor}.pkl', 'wb') as f:
        pickle.dump(G, f)
    return G

##1~3층 JSON 기반 그래프 로드

In [ ]:
G1 = load_graph_from_json('/content/drive/MyDrive/1f.json', 1)
G2 = load_graph_from_json('/content/drive/MyDrive/2f.json', 2)
G3 = load_graph_from_json('/content/drive/MyDrive/3f.json', 3)

# 저장
with open('/content/drive/MyDrive/G1.pkl', 'wb') as f:
    pickle.dump(G1, f)
with open('/content/drive/MyDrive/G2.pkl', 'wb') as f:
    pickle.dump(G2, f)
with open('/content/drive/MyDrive/G3.pkl', 'wb') as f:
    pickle.dump(G3, f)

##그래프 통합 및 층 간 연결

In [ ]:
# 로드 (저장된 파일 사용)
with open('/content/drive/MyDrive/G1.pkl', 'rb') as f:
    G1 = pickle.load(f)
with open('/content/drive/MyDrive/G2.pkl', 'rb') as f:
    G2 = pickle.load(f)
with open('/content/drive/MyDrive/G3.pkl', 'rb') as f:
    G3 = pickle.load(f)
with open('/content/drive/MyDrive/G4.pkl', 'rb') as f:
    G4 = pickle.load(f)
with open('/content/drive/MyDrive/G5.pkl', 'rb') as f:
    G5 = pickle.load(f)

G = nx.compose_all([G1, G2, G3, G4, G5])

# 층 간 연결 추가
from collections import defaultdict
stair_nodes = [node for node in G.nodes if G.nodes[node]['type'] == 'Stair']
elevator_nodes = [node for node in G.nodes if G.nodes[node]['type'] == 'Elevator']

stair_groups = defaultdict(list)
for node in stair_nodes:
    name = G.nodes[node]['name']
    parts = name.split('_')
    if len(parts) == 2 and parts[1].endswith('f'):
        stair_id = parts[0].replace('stair', '')
        floor = int(parts[1][:-1])
        stair_groups[stair_id].append((node, floor))

for group in stair_groups.values():
    group.sort(key=lambda x: x[1])
    for i in range(len(group) - 1):
        node1, node2 = group[i][0], group[i+1][0]
        G.add_edge(node1, node2, weight=50)

elevator_groups = defaultdict(list)
for node in elevator_nodes:
    name = G.nodes[node]['name']
    parts = name.split('_')
    if len(parts) == 2 and parts[1].endswith('f'):
        elevator_id = parts[0].replace('elevator', '')
        floor = int(parts[1][:-1])
        elevator_groups[elevator_id].append((node, floor))

for group in elevator_groups.values():
    group.sort(key=lambda x: x[1])
    for i in range(len(group) - 1):
        node1, node2 = group[i][0], group[i+1][0]
        G.add_edge(node1, node2, weight=20)

# 저장
with open('/content/drive/MyDrive/final_G.pkl', 'wb') as f:
    pickle.dump(G, f)

##경로 탐색 함수 정의 및 실행

In [ ]:
# 경로 탐색 함수 정의
with open('/content/drive/MyDrive/final_G.pkl', 'rb') as f:
    G = pickle.load(f)

room_to_node = {G.nodes[node]['name']: node for node in G.nodes if G.nodes[node]['type'] == 'Room'}

def find_path(start_room, end_room):
    start_node = room_to_node.get(start_room)
    end_node = room_to_node.get(end_room)
    if start_node and end_node:
        path = nx.shortest_path(G, source=start_node, target=end_node, weight='weight')
        simplified_path = [node for node in path if G.nodes[node]['type'] != 'Corridor']
        path_names = [G.nodes[node]['name'] for node in simplified_path]
        return " -> ".join(path_names)
    return "강의실을 찾을 수 없습니다."

In [ ]:
# 사용자 입력 및 경로 출력
start_room = input("시작 강의실 입력 (예: 27508A): ")
end_room = input("도착 강의실 입력 (예: 25108): ")
result = find_path(start_room, end_room)
print(result)

#Mask R-CNN 사용 방법

## 환경 설정

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html
!pip install opencv-python-headless pytesseract networkx
!apt-get update
!apt-get install -y tesseract-ocr libtesseract-dev
from google.colab import drive
drive.mount('/content/drive')

##학습 데이터 준비

In [ ]:
import cv2
import numpy as np
import json
import pytesseract
from detectron2.structures import BoxMode
import os

def get_floorplan_dicts(img_dir, json_dir):
    dataset_dicts = []
    categories = {"Room": 0, "Corridor": 1, "Stair": 2, "Elevator": 3, "Restroom": 4, "Door": 5}

    for floor in [1, 2, 3]:
        img_path = os.path.join(img_dir, f"{floor}.jpg")
        json_path = os.path.join(json_dir, f"{floor}.json")
        image = cv2.imread(img_path)
        height, width = image.shape[:2]

        with open(json_path, 'r') as f:
            data = json.load(f)

        # 이미지 전처리: 벽과 복도 식별
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 노드와 컨투어 매핑
        annotations = []
        for node in [item for item in data if "id" in item]:
            x, y = node['x'], node['y']
            for contour in contours:
                if cv2.pointPolygonTest(contour, (x, y), False) >= 0:
                    x, y, w, h = cv2.boundingRect(contour)
                    poly = contour.reshape(-1, 2).tolist()
                    annotations.append({
                        "bbox": [x, y, x+w, y+h],
                        "bbox_mode": BoxMode.XYXY_ABS,
                        "segmentation": [poly],
                        "category_id": categories[node['type']],
                        "id": node['id']
                    })
                    break

        # 텍스트 추출 및 매핑
        text_data = pytesseract.image_to_data(image, lang='kor', output_type=pytesseract.Output.DICT)
        for i, text in enumerate(text_data['text']):
            if text.strip():
                x, y = text_data['left'][i], text_data['top'][i]
                for ann in annotations:
                    bbox = ann['bbox']
                    if bbox[0] <= x <= bbox[2] and bbox[1] <= y <= bbox[3]:
                        ann['room_label'] = text
                        break

        dataset_dicts.append({
            "file_name": img_path,
            "image_id": floor,
            "height": height,
            "width": width,
            "annotations": annotations
        })

    # 데이터셋 저장
    with open('/content/drive/MyDrive/floorplan_dataset.json', 'w') as f:
        json.dump(dataset_dicts, f)
    return dataset_dicts

# 데이터셋 등록
from detectron2.data import DatasetCatalog, MetadataCatalog
DatasetCatalog.register("floorplan_train", lambda: get_floorplan_dicts('/content/drive/MyDrive/images', '/content/drive/MyDrive/json'))
MetadataCatalog.get("floorplan_train").set(thing_classes=["Room", "Corridor", "Stair", "Elevator", "Restroom", "Door"])

## Mask R-CNN 학습

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("floorplan_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# 모델 저장
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/model_final.pth'
with open('/content/drive/MyDrive/model_config.yaml', 'w') as f:
    f.write(cfg.dump())

## 그래프 생성

In [ ]:
from detectron2.engine import DefaultPredictor
import networkx as nx
import pickle

def build_graph_from_image(image_path, floor, cfg):
    image = cv2.imread(image_path)
    predictor = DefaultPredictor(cfg)
    outputs = predictor(image)
    instances = outputs["instances"].to("cpu")

    text_data = pytesseract.image_to_data(image, lang='kor', output_type=pytesseract.Output.DICT)
    G = nx.Graph()

    # 노드 추가
    for i, (mask, cls) in enumerate(zip(instances.pred_masks, instances.pred_classes)):
        mask = mask.numpy()
        M = cv2.moments(mask.astype(np.uint8))
        cx = int(M['m10'] / M['m00']) if M['m00'] != 0 else 0
        cy = int(M['m01'] / M['m00']) if M['m00'] != 0 else 0
        node_id = f"{floor}_{i}"
        class_name = ["Room", "Corridor", "Stair", "Elevator", "Restroom", "Door"][cls]

        # 텍스트 매핑
        name = f"{class_name}_{i}"
        for j, text in enumerate(text_data['text']):
            if text.strip():
                x = text_data['left'][j] + text_data['width'][j] // 2
                y = text_data['top'][j] + text_data['height'][j] // 2
                if mask[y, x]:
                    name = text
                    break
        G.add_node(node_id, name=name, type=class_name, x=cx, y=cy, floor=floor)

    # 복도 마스크로 그리드 노드 생성
    corridor_mask = np.zeros(image.shape[:2], dtype=bool)
    for i, (mask, cls) in enumerate(zip(instances.pred_masks, instances.pred_classes)):
        if cls == 1:  # Corridor
            corridor_mask |= mask.numpy()

    grid_spacing = 20
    height, width = image.shape[:2]
    grid_nodes = []
    for i in range(0, height, grid_spacing):
        for j in range(0, width, grid_spacing):
            if corridor_mask[i, j]:
                node_id = f"{floor}_grid_{i}_{j}"
                G.add_node(node_id, name='Corridor', type='Corridor', x=j, y=i, floor=floor)
                grid_nodes.append(node_id)

    # 그리드 노드 연결
    for node in grid_nodes:
        x = G.nodes[node]['x']
        y = G.nodes[node]['y']
        neighbors = [(x + grid_spacing, y), (x - grid_spacing, y), (x, y + grid_spacing), (x, y - grid_spacing)]
        for nx, ny in neighbors:
            if 0 <= ny < height and 0 <= nx < width and corridor_mask[ny, nx]:
                neighbor_id = f"{floor}_grid_{ny}_{nx}"
                if neighbor_id in G.nodes:
                    distance = np.sqrt((x - nx)**2 + (y - ny)**2)
                    G.add_edge(node, neighbor_id, weight=distance)

    # 방, 문, 계단 등과 복도 연결
    for node in G.nodes:
        if G.nodes[node]['type'] in ['Room', 'Door', 'Stair', 'Elevator', 'Restroom']:
            x, y = G.nodes[node]['x'], G.nodes[node]['y']
            min_distance = float('inf')
            nearest_grid = None
            for grid_node in grid_nodes:
                gx, gy = G.nodes[grid_node]['x'], G.nodes[grid_node]['y']
                distance = np.sqrt((x - gx)**2 + (y - gy)**2)
                if distance < min_distance:
                    min_distance = distance
                    nearest_grid = grid_node
            if nearest_grid:
                G.add_edge(node, nearest_grid, weight=min_distance)

    # 그래프 저장
    with open(f'/content/drive/MyDrive/G{floor}.pkl', 'wb') as f:
        pickle.dump(G, f)
    return G

# 4~5층 그래프 생성
cfg = get_cfg()
cfg.merge_from_file('/content/drive/MyDrive/model_config.yaml')
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/model_final.pth'
G4 = build_graph_from_image('/content/drive/MyDrive/4.jpg', 4, cfg)
G5 = build_graph_from_image('/content/drive/MyDrive/5.jpg', 5, cfg)

## 그래프 통합 + 층간 연결

In [ ]:
import pickle
from collections import defaultdict

# 그래프 로드
with open('/content/drive/MyDrive/G1.pkl', 'rb') as f:
    G1 = pickle.load(f)
with open('/content/drive/MyDrive/G2.pkl', 'rb') as f:
    G2 = pickle.load(f)
with open('/content/drive/MyDrive/G3.pkl', 'rb') as f:
    G3 = pickle.load(f)
with open('/content/drive/MyDrive/G4.pkl', 'rb') as f:
    G4 = pickle.load(f)
with open('/content/drive/MyDrive/G5.pkl', 'rb') as f:
    G5 = pickle.load(f)

G = nx.compose_all([G1, G2, G3, G4, G5])

# 층 간 연결
stair_nodes = [node for node in G.nodes if G.nodes[node]['type'] == 'Stair']
elevator_nodes = [node for node in G.nodes if G.nodes[node]['type'] == 'Elevator']

stair_groups = defaultdict(list)
for node in stair_nodes:
    name = G.nodes[node]['name']
    parts = name.split('_')
    if len(parts) == 2 and parts[1].endswith('f'):
        stair_id = parts[0].replace('stair', '')
        floor = int(parts[1][:-1])
        stair_groups[stair_id].append((node, floor))

for group in stair_groups.values():
    group.sort(key=lambda x: x[1])
    for i in range(len(group) - 1):
        node1, node2 = group[i][0], group[i+1][0]
        G.add_edge(node1, node2, weight=50)

elevator_groups = defaultdict(list)
for node in elevator_nodes:
    name = G.nodes[node]['name']
    parts = name.split('_')
    if len(parts) == 2 and parts[1].endswith('f'):
        elevator_id = parts[0].replace('elevator', '')
        floor = int(parts[1][:-1])
        elevator_groups[elevator_id].append((node, floor))

for group in elevator_groups.values():
    group.sort(key=lambda x: x[1])
    for i in range(len(group) - 1):
        node1, node2 = group[i][0], group[i+1][0]
        G.add_edge(node1, node2, weight=20)

# 최종 그래프 저장
with open('/content/drive/MyDrive/final_G.pkl', 'wb') as f:
    pickle.dump(G, f)

## 경로 탐색

In [ ]:
with open('/content/drive/MyDrive/final_G.pkl', 'rb') as f:
    G = pickle.load(f)

room_to_node = {G.nodes[node]['name']: node for node in G.nodes if G.nodes[node]['type'] == 'Room'}

def find_path(start_room, end_room):
    start_node = room_to_node.get(start_room)
    end_node = room_to_node.get(end_room)
    if start_node and end_node:
        path = nx.shortest_path(G, source=start_node, target=end_node, weight='weight')
        simplified_path = [node for node in path if G.nodes[node]['type'] != 'Corridor']
        path_names = [G.nodes[node]['name'] for node in simplified_path]
        return " -> ".join(path_names)
    return "강의실을 찾을 수 없습니다."

start_room = input("시작 강의실 입력 (예: 27508A): ")
end_room = input("도착 강의실 입력 (예: 25108): ")
result = find_path(start_room, end_room)
print(result)